# Experiment 2.2: Specific concept intervention

In the [1.x series](/README.md#m1-preliminary-experiments-with-color) of experiments (milestone 1), we validated our ideas for imposing structure on latent space. With only weak supervision, we guided a simple RGB autoencoder to use the color wheel for its latent representations. In this series, we'll try to inhibit and even delete certain concepts from the model.

To start, let's take one of the earlier experiments and see what happens when we suppress activations that align with _red_.

## Hypothesis

We've structured the latent space so red is located at $[1,0,0,0]$. If we suppress or redirect activations close to that vector, model performance on near-red colors should drop, while other colors remain mostly unaffected.

In [1]:
# Basic setup: Logging, Experiment (Modal)
from __future__ import annotations
import logging

import modal

from infra.requirements import freeze, project_packages
from mini.experiment import Experiment
from utils.logging import SimpleLoggingConfig

logging_config = (
    SimpleLoggingConfig()
    .info('notebook', 'utils', 'mini', 'ex_color')
    .error('matplotlib.axes')  # Silence warnings about set_aspect
)
logging_config.apply()

# ID for tagging assets
nbid = '2.2'
# This is the logger for this notebook
log = logging.getLogger(f'notebook.{nbid}')
experiment_name = f'ex-color-{nbid}'

run = Experiment(experiment_name)
run.image = modal.Image.debian_slim().pip_install(*freeze(all=True)).add_local_python_source(*project_packages())
run.before_each(logging_config.apply)
None  # prevent auto-display of this cell

## Regularizers

Like Ex 1.7:

- **Anchor:** pins `red` to $(1,0,0,0)$
- **Separate:** angular repulsion to reduce global clumping (applied within each batch)
- **Planarity:** pulls vibrant hues to the $[0, 1]$ plane
- **Unitarity:** pulls all embeddings to the surface of the unit hypersphere, i.e. it makes the embedding vectors have unit length. There are two terms: one that affects all colors equally, and another that just operates on the vibrant colors (because they seemed to need a little more help).

In [2]:
import torch

from mini.temporal.dopesheet import Dopesheet
from ex_color.loss import Anchor, Separate, Planarity, Unitarity, RegularizerConfig
from ex_color.model import ColorMLP
from ex_color.training import TrainingModule

RED = (1, 0, 0, 0)

ALL_REGULARIZERS = [
    RegularizerConfig(
        name='reg-anchor',
        compute_loss_term=Anchor(torch.tensor(RED, dtype=torch.float32)),
        label_affinities={'red': 1.0},
        layer_affinities=['encoder'],
    ),
    RegularizerConfig(
        name='reg-separate',
        compute_loss_term=Separate(power=10.0, shift=False),
        label_affinities=None,
        layer_affinities=['encoder'],
    ),
    RegularizerConfig(
        name='reg-planar',
        compute_loss_term=Planarity(),
        label_affinities={'vibrant': 1.0},
        layer_affinities=['encoder'],
    ),
    RegularizerConfig(
        name='reg-unit-v',
        compute_loss_term=Unitarity(),
        label_affinities={'vibrant': 1.0},
        layer_affinities=['encoder'],
    ),
    RegularizerConfig(
        name='reg-unit',
        compute_loss_term=Unitarity(),
        label_affinities=None,
        layer_affinities=['encoder'],
    ),
]

## Data

Data is the same as last time:
- Train: an HSV cube (of RGB values)
- Test: an RGB cube

In [3]:
from functools import partial
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import numpy as np

from ex_color.data.color_cube import ColorCube
from ex_color.data.cube_sampler import vibrancy
from ex_color.data.cyclic import arange_cyclic
from ex_color.labelling import collate_with_generated_labels


def prep_data() -> tuple[DataLoader, Tensor]:
    """
    Prepare data for training.

    Returns: (train, val)
    """
    hsv_cube = ColorCube.from_hsv(
        h=arange_cyclic(step_size=10 / 360),
        s=np.linspace(0, 1, 10),
        v=np.linspace(0, 1, 10),
    )
    hsv_tensor = torch.tensor(hsv_cube.rgb_grid.reshape(-1, 3), dtype=torch.float32)
    vibrancy_tensor = torch.tensor(vibrancy(hsv_cube).flatten(), dtype=torch.float32)
    hsv_dataset = TensorDataset(hsv_tensor, vibrancy_tensor)

    labeller = partial(
        collate_with_generated_labels,
        soft=False,  # Use binary labels (stochastic) to simulate the labelling of internet text
        scale={'red': 0.5, 'vibrant': 0.5},
    )
    # Desaturated and dark colors are over-represented in the cube, so we use a weighted sampler to balance them out
    hsv_loader = DataLoader(
        hsv_dataset,
        batch_size=64,
        num_workers=2,
        sampler=WeightedRandomSampler(
            weights=hsv_cube.bias.flatten().tolist(),
            num_samples=len(hsv_dataset),
            replacement=True,
        ),
        collate_fn=labeller,
    )

    rgb_cube = ColorCube.from_rgb(
        r=np.linspace(0, 1, 8),
        g=np.linspace(0, 1, 8),
        b=np.linspace(0, 1, 8),
    )
    rgb_tensor = torch.tensor(rgb_cube.rgb_grid.reshape(-1, 3), dtype=torch.float32)
    return hsv_loader, rgb_tensor

## Training

_Unlike_ earlier experiments, we've switched over to use PyTorch Lightning instead of our custom training loop. We also tried porting to Catalyst and Ignite, but we found that Lightning was the closest match to the shape that our training code had evolved into.

Functionally, not much has changed at this point, but now we should be able to take advantage of things like [Lightning's distributed processing support](https://lightning.ai/docs/pytorch/stable/api_references.html#strategies).

We have also switched to using Modal for remote compute, and Weights and Biases for experiment tracking. We also tried running our own Aim experiment tracker instance. It worked, but it was slow. We're not sure why; maybe we just hadn't configured the storage or networking properly. If you're curious, check out the [`aim` tag in the Git history](https://github.com/z0u/ex-color-transformer/tree/aim).

In [4]:
import wandb
from ex_color.inference import InferenceModule
from ex_color.intervention.intervention import InterventionConfig

wandb_api_key = wandb.Api().api_key


# @run.thither
async def train(
    dopesheet: Dopesheet,
    regularizers: list[RegularizerConfig],
) -> ColorMLP:
    """Train the model with the given dopesheet and variant."""
    import lightning as L
    from lightning.pytorch.loggers import WandbLogger

    from ex_color.seed import set_deterministic_mode

    from utils.progress.lightning import LightningProgress

    log.info(f'Training with: {[r.name for r in regularizers]}')

    seed = 0
    set_deterministic_mode(seed)

    hsv_loader, _ = prep_data()

    model = ColorMLP(4)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    log.debug(f'Model initialized with {total_params:,} trainable parameters.')

    training_module = TrainingModule(model, dopesheet, torch.nn.MSELoss(), regularizers)

    wandb.login(key=wandb_api_key)
    logger = WandbLogger(experiment_name, project='ex-color-transformer')

    trainer = L.Trainer(
        max_steps=len(dopesheet),
        callbacks=[
            LightningProgress(),
        ],
        enable_checkpointing=False,
        enable_model_summary=False,
        # enable_progress_bar=True,
        logger=logger,
    )

    print(f'max_steps: {len(dopesheet)}, hsv_loader length: {len(hsv_loader)}')

    # Train the model
    trainer.fit(training_module, hsv_loader)
    # This is only a small model, so it's OK to return it rather than storing and loading a checkpoint remotely
    return model


async with run():
    model = await train(Dopesheet.from_csv(f'./ex-{nbid}-dopesheet.csv'), ALL_REGULARIZERS)

I 4.6 no.2.2:  Training with: ['reg-anchor', 'reg-separate', 'reg-planar', 'reg-unit-v', 'reg-unit']


INFO: Seed set to 0


I 4.6 li.fa.ut.se:Seed set to 0
I 4.7 ex.se:   PyTorch set to deterministic mode
I 4.7 ex.se:   PyTorch set to deterministic mode


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/vscode/.netrc
wandb: Appending key for api.wandb.ai to your netrc file: /home/vscode/.netrc
wandb: Currently logged in as: z0r to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: z0r to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
INFO: GPU available: False, used: False
INFO: GPU available: False, used: False


I 5.3 li.py.ut.ra:GPU available: False, used: False


...

I 5.3 li.py.ut.ra:HPU available: False, using: 0 HPUs
max_steps: 2001, hsv_loader length: 57
max_steps: 2001, hsv_loader length: 57


Training: 100.0% [2001/2001]

Starting phase: Train


INFO: `Trainer.fit` stopped: `max_steps=2001` reached.


I 18.0 li.py.ut.ra:`Trainer.fit` stopped: `max_steps=2001` reached.


## Inference ('test-time')

We wrap the model that we trained above in an `InferenceModule`, which knows how to apply our interventions.

In [5]:
# @run.thither
async def infer(
    model: ColorMLP,
    interventions: list[InterventionConfig],
    test_data: Tensor,
) -> Tensor:
    """Run inference with the given model and interventions."""
    import lightning as L

    inference_module = InferenceModule(model, interventions)
    trainer = L.Trainer(
        enable_checkpointing=False,
        enable_model_summary=False,
        enable_progress_bar=True,
    )
    reconstructed_colors_batches = trainer.predict(
        inference_module,
        DataLoader(
            TensorDataset(test_data.reshape((-1, 3))),
            batch_size=64,
            collate_fn=lambda batch: torch.stack([row[0] for row in batch], 0),
        ),
    )
    assert reconstructed_colors_batches is not None
    # Flatten the list of batches to a single list of tensors
    reconstructed_colors = [item for batch in reconstructed_colors_batches for item in batch]
    # Reshape to match input
    return torch.cat(reconstructed_colors).reshape(test_data.shape)

Let's see how well the model reconstructs colors _without_ any interventions.

In [6]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

from ex_color.vis import plot_colors
from utils.nb import displayer_img
from utils.plt import configure_matplotlib


configure_matplotlib()


hsv_cube = ColorCube.from_hsv(
    h=arange_cyclic(step_size=1 / 24),
    s=np.linspace(0, 1, 4),
    v=np.linspace(0, 1, 8),
).permute('svh')
x_hsv = torch.tensor(hsv_cube.rgb_grid, dtype=torch.float32)
clear_output()
with displayer_img(f'ex-{nbid}-true-colors.png') as show:
    fig = plot_colors(hsv_cube, title='True colors', colors=x_hsv.numpy())
    show(fig)
    plt.close(fig)

Image

In [7]:
from IPython.display import clear_output


y_hsv = await infer(model, [], x_hsv)
clear_output()
with displayer_img(f'ex-{nbid}-pred-colors-no-intervention.png') as show:
    fig = plot_colors(
        hsv_cube,
        title='Predicted colors without intervention',
        colors=y_hsv.numpy(),
        colors_compare=x_hsv.numpy(),
    )
    show(fig)
    plt.close(fig)

Image

That's pretty good: as expected, the reconstructed colors (predictions) _look_ almost the same as the true colors. Visually, the main differences I can see are:

- Fully saturated colors ($s=1$) show some bleeding of green, red, and hot pink into neighboring hues
- Fully desaturated colors ($s=0$) show some hint of being slightly off-gray, i.e. some saturation has crept in.

Sense-check: let's look at the latent space too. We'll use an RGB cube as input for this instead of the HSV cube used above, because it gives a more regular distribution of points — which will be useful to see whether the intervention changes the point density.

In [8]:
# Capture encoder activations (latents) without interventions
import torch
import numpy as np

from ex_color.inference import InferenceModule


# Build a tiny helper that runs predict while capturing latents from 'encoder'
async def infer_with_latent_capture(
    model: ColorMLP, interventions: list[InterventionConfig], test_data: Tensor, layer_name: str = 'encoder'
) -> tuple[Tensor, Tensor]:
    module = InferenceModule(model, interventions, capture_layers=[layer_name])
    import lightning as L

    trainer = L.Trainer(enable_checkpointing=False, enable_model_summary=False, enable_progress_bar=False)
    batches = trainer.predict(
        module,
        DataLoader(
            TensorDataset(test_data.reshape((-1, 3))),
            batch_size=64,
            collate_fn=lambda batch: torch.stack([row[0] for row in batch], 0),
        ),
    )
    assert batches is not None
    preds = [item for batch in batches for item in batch]
    y = torch.cat(preds).reshape(test_data.shape)
    # Read captured activations as a flat [N, D] tensor
    latents = module.read_captured(layer_name)
    return y, latents

In [9]:
# Plotting utilities for latent slices (axis-aligned 2D with depth ordering)
from typing import cast, Sequence

from matplotlib.typing import ColorType
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
from numpy.typing import NDArray
import torch

from ex_color.data.color_cube import ColorCube
from utils.plt import hide_decorations


from typing import cast, Sequence
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.typing import ColorType
from mpl_toolkits.mplot3d.art3d import Line3DCollection, Poly3DCollection
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
from numpy.typing import NDArray
import torch

from ex_color.data.color_cube import ColorCube
from utils.plt import hide_decorations


def draw_latent_3d(
    ax: Axes3D,
    latents_3d: np.ndarray,
    colors_rgb: np.ndarray,
    colors_edge_rgb: np.ndarray | None = None,
    *,
    dot_size: float = 10.0,
    alpha: float = 1.0,
):
    if colors_edge_rgb is None:
        colors_edge_rgb = colors_rgb

    scatter = ax.scatter(
        latents_3d[:, 0],
        latents_3d[:, 1],
        latents_3d[:, 2],  # type: ignore
        c=colors_rgb,
        edgecolors=cast(Sequence[ColorType], colors_edge_rgb),
        linewidths=0.01 * dot_size,
        s=dot_size,  # type: ignore
        alpha=alpha,
    )

    # Set equal aspect ratio for all axes
    max_range = np.abs(latents_3d).max()
    ax.set_xlim([-max_range * 1.1, max_range * 1.1])
    ax.set_ylim([-max_range * 1.1, max_range * 1.1])
    ax.set_zlim([-max_range * 1.1, max_range * 1.1])

    return {'scatter': scatter}


def draw_circle_3d(
    ax: Axes3D,
    r=1,
    verts=100,
    color: ColorType | None = None,
    alpha: float = 1.0,
    zorder: float | None = None,
):
    theta = np.linspace(0, 2 * np.pi, verts)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    z = np.full_like(x, 0)

    # Create vertices for the filled circle
    circle_verts = [list(zip(x, y, z, strict=True))]
    poly = Poly3DCollection(circle_verts, facecolors=color, alpha=alpha, zorder=-10)
    ax.add_collection3d(poly)
    return poly


def _edge_index_chains(shape: tuple[int, int, int]) -> list[np.ndarray]:
    """
    Compute index chains for each cube edge at full resolution.
    (This function stays the same - it's just computing connectivity)
    """
    edges: list[np.ndarray] = []
    dims = tuple(int(x) for x in shape)
    for k in range(3):
        fixed_axes = [a for a in (0, 1, 2) if a != k]
        for f0 in (0, dims[fixed_axes[0]] - 1):
            for f1 in (0, dims[fixed_axes[1]] - 1):
                n_k = dims[k]
                multi = np.zeros((n_k, 3), dtype=int)
                coords = [0, 0, 0]
                coords[fixed_axes[0]] = f0
                coords[fixed_axes[1]] = f1
                for t in range(n_k):
                    coords[k] = t
                    multi[t] = coords
                idx_chain = cast(NDArray, np.ravel_multi_index(multi.T, dims))
                edges.append(idx_chain)
    return edges


def _draw_edge_mesh_3d(
    ax: Axes3D,
    latents_3d: np.ndarray,
    point_colors: np.ndarray,
    edge_index_chains: list[np.ndarray],
    *,
    alpha: float = 0.95,
    linewidth: float = 0.6,
):
    """
    Draw 3D edge mesh. Much simpler - no manual depth sorting!
    Axes3D handles all the depth complexity for us.
    """
    segments = []
    colors = []

    for chain in edge_index_chains:
        if len(chain) < 2:
            continue

        pts_3d = latents_3d[chain]  # (n, 3)
        cols = point_colors[chain]  # (n, 3)

        # Create line segments between consecutive points
        for i in range(len(pts_3d) - 1):
            segments.append([pts_3d[i], pts_3d[i + 1]])
            # Average color between the two endpoints
            colors.append(0.5 * (cols[i] + cols[i + 1]))

    if segments:
        lc = Line3DCollection(segments, colors=colors, linewidths=linewidth, alpha=alpha)
        ax.add_collection3d(lc)


def plot_latent_grid_3d(
    cube: ColorCube,
    latents: torch.Tensor,
    colors: torch.Tensor,
    colors_edge: torch.Tensor | None = None,
    *,
    dims: list[tuple[int, int, int]],
    title: str | None = None,
    figsize_per_plot: tuple[float, float] = (6, 6),
):
    """
    Plot 4D+ latent data as true 3D visualizations.

    Args:
        latents: Shape [..., D] where D >= 3
        dims: List of (i, j, k) triplets specifying which latent dimensions to use for x, y, z
    """
    lat_np = latents.detach().cpu().numpy()
    col_np = colors.detach().cpu().reshape(-1, colors.shape[-1]).numpy()
    col_edge_np = (
        colors_edge.detach().cpu().reshape(-1, colors.shape[-1]).numpy() if colors_edge is not None else col_np
    )

    n = len(dims)
    cols = min(3, n)  # Max 3 columns
    rows = int(np.ceil(n / cols))

    # Precompute edge connectivity
    assert len(cube.shape) == 3
    edge_chains = _edge_index_chains(cube.shape)

    fig = plt.figure(figsize=(figsize_per_plot[0] * cols, figsize_per_plot[1] * rows))

    for idx, (i, j, k) in enumerate(dims):
        # Create 3D subplot
        ax = cast(Axes3D, fig.add_subplot(rows, cols, idx + 1, axes_class=Axes3D))

        # Extract 3D coordinates
        lat_3d = lat_np[:, [i, j, k]]

        draw_circle_3d(ax, color='#111')

        # Draw edges and points (automatically depth-sorted by Axes3D)
        _draw_edge_mesh_3d(ax, lat_3d, col_edge_np, edge_chains, alpha=0.7, linewidth=2.0)
        draw_latent_3d(ax, lat_3d, col_np, colors_edge_rgb=col_edge_np, alpha=1.0, dot_size=10)

        # Clean up the 3D axes
        hide_decorations(ax)
        ax.set_title(f'Latent dims ({i},{j},{k})')
        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-1.2, 1.2)
        ax.view_init(elev=90, azim=-90)
        ax.set_proj_type('ortho')

    if title:
        fig.suptitle(title)

    plt.tight_layout()
    return fig


# Visualize latents without interventions
from utils.nb import displayer_img


rgb_cube = ColorCube.from_rgb(
    r=np.linspace(0, 1, 20),
    g=np.linspace(0, 1, 20),
    b=np.linspace(0, 1, 20),
)
x_rgb = torch.tensor(rgb_cube.rgb_grid, dtype=torch.float32)

y_rgb, h_rgb = await infer_with_latent_capture(model, [], x_rgb, 'encoder')
clear_output()
with displayer_img(f'ex-{nbid}-latents-no-intervention.png') as show:
    fig = plot_latent_grid_3d(
        rgb_cube, h_rgb, y_rgb, x_rgb, title='Latents · no intervention', dims=[(1, 0, 2), (1, 2, 0), (2, 3, 0)]
    )
    show(fig)
    plt.close(fig)

Image

The looks reasonable: similar to Ex 1.7, the latent space shows a color wheel in the first two axes with red near the top.

## Suppression

Now that we have our model, let's try suppressing _red_. We'll use the `Suppression` function developed in [Ex 2.1](./ex-2.1-intervention-lobe.ipynb).

In [10]:
import importlib
from math import cos, pi

import torch

import ex_color.intervention
import ex_color.intervention.bounded_falloff

importlib.reload(ex_color.intervention.bounded_falloff)
importlib.reload(ex_color.intervention)
from ex_color.intervention import BoundedFalloff, InterventionConfig, Suppression


suppression = InterventionConfig(
    Suppression(
        torch.tensor(RED, dtype=torch.float32),  # Constant!
        BoundedFalloff(
            0,  # within 60°
            1,  # completely squash fully-aligned vectors
            2,  # soft rim, sharp hub
        ),
    ),
    ['encoder'],  # output of encoder is the bottleneck
)

y_hsv = await infer(model, [suppression], x_hsv)
clear_output()
with displayer_img(f'ex-{nbid}-pred-colors-suppression.png') as show:
    fig = plot_colors(
        hsv_cube,
        title='Predicted colors with suppression',
        colors=y_hsv.numpy(),
        colors_compare=x_hsv.numpy(),
    )
    show(fig)
    plt.close(fig)

Image

In [11]:
# Capture latents with suppression
y_rgb, h_rgb = await infer_with_latent_capture(model, [suppression], x_rgb, 'encoder')
clear_output()
with displayer_img(f'ex-{nbid}-latents-suppression.png') as show:
    fig = plot_latent_grid_3d(
        rgb_cube, h_rgb, y_rgb, x_rgb, title='Latents · suppression', dims=[(1, 0, 2), (1, 2, 0), (2, 3, 0)]
    )
    show(fig)
    plt.close(fig)

Image

## Repulsion

In [12]:
from math import cos, pi

import torch

from ex_color.intervention import FastBezierMapper, InterventionConfig, Repulsion

repulsion = InterventionConfig(
    Repulsion(
        torch.tensor([1, 0, 0, 0], dtype=torch.float32),  # Constant!
        FastBezierMapper(
            0,  # Constrain effect to within 60° cone
            cos(pi / 3),  # Create 30° hole (negative cone) around concept vector
        ),
    ),
    ['encoder'],
)

y_hsv = await infer(model, [repulsion], x_hsv)
clear_output()
with displayer_img(f'ex-{nbid}-pred-colors-repulsion.png') as show:
    fig = plot_colors(
        hsv_cube,
        title='Predicted colors with repulsion',
        colors=y_hsv.numpy(),
        colors_compare=x_hsv.numpy(),
    )
    show(fig)
    plt.close(fig)

Image

In [13]:
# Capture latents with repulsion
y_rgb, h_rgb = await infer_with_latent_capture(model, [repulsion], x_rgb, 'encoder')
clear_output()
with displayer_img(f'ex-{nbid}-latents-repulsion.png') as show:
    fig = plot_latent_grid_3d(
        rgb_cube, h_rgb, y_rgb, x_rgb, title='Latents · repulsion', dims=[(1, 0, 2), (1, 2, 0), (2, 3, 0)]
    )
    show(fig)
    plt.close(fig)

Image